In [1]:
! pip install pymupdf
! pip install PyPDF2
! pip install sentence-transformers
! pip install faiss-cpu
! pip install transformers
! pip install gradio

In [2]:
from PyPDF2 import PdfReader
import fitz
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re
import requests
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import gradio as gr

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# 1. Text Extraction

In [3]:
def preprocess(text):
    """
    Preprocess the input text by removing extra whitespace and newline characters.

    Parameters:
    text (str): The input text to be preprocessed.

    Returns:
    str: The preprocessed text with newline characters replaced by spaces and
         multiple spaces reduced to a single space.
    """
    text = text.replace('\n', ' ')
    text = re.sub('\s+', ' ', text)
    return text

In [4]:
def pdf_to_text(path, start_page=1, end_page=None):
    """
    Extract text from specified pages of a PDF file.

    This function opens a PDF file located at the specified `path`, extracts the text
    from pages ranging from `start_page` to `end_page` (inclusive), preprocesses the text,
    and returns a list of text strings, one for each page.

    Parameters:
    path (str): The path to the PDF file.
    start_page (int, optional): The starting page number (1-based index) from which
                                 to start extracting text. Defaults to 1.
    end_page (int, optional): The ending page number (1-based index) up to which
                               text should be extracted. If None, text is extracted
                               until the last page. Defaults to None.

    Returns:
    list of str: A list where each element is a string containing the preprocessed
                 text from a corresponding page of the PDF.
    """
    doc = fitz.open(path)
    total_pages = doc.page_count

    if end_page is None:
        end_page = total_pages

    text_list = []

    for i in range(start_page - 1, end_page):
        text = doc.load_page(i).get_text("text")
        text = preprocess(text)
        text_list.append(text)

    doc.close()
    return text_list

In [5]:
def text_to_chunks(texts, word_length=50, overlap=10, start_page=1):
    """
    Split text from multiple pages into chunks of specified word length with overlap.

    This function takes a list of text strings (one for each page), splits each text
    into chunks of a specified word length with a specified overlap, and formats these
    chunks with page numbers for easy reference. If a chunk on a page is smaller than
    the specified word length and there are more pages, it concatenates this chunk
    with the beginning of the following page's text.

    Parameters:
    texts (list of str): A list where each element is a string representing the text
                         of a page.
    word_length (int, optional): The maximum number of words per chunk. Defaults to 50.
    overlap (int, optional): The number of words to overlap between chunks. Defaults to 10.
    start_page (int, optional): The starting page number to use for formatting the page
                                number in the chunk. Defaults to 1.

    Returns:
    list of str: A list of text chunks where each chunk is a string formatted with the
                 page number and the chunked text content.
    """
    text_toks = [t.split(' ') for t in texts]
    chunks = []

    for idx, words in enumerate(text_toks):
        i = 0
        while i < len(words):
            # Create a chunk of the specified length
            chunk = words[i:i + word_length]

            # If this is the last chunk and it's shorter than word_length, check if we need to merge with the next page
            if (i + word_length) > len(words) and len(chunk) < word_length and (len(text_toks) != (idx + 1)):
                text_toks[idx + 1] = chunk + text_toks[idx + 1]
                break

            # Join the chunk into a string and format it with the page number
            chunk = ' '.join(chunk).strip()
            chunk = f'[Page no. {idx + start_page}]' + ' ' + '"' + chunk + '"'
            chunks.append(chunk)

            # Move the index forward by the word length minus the overlap
            i += word_length - overlap

    return chunks


In [6]:
# URL of the PDF file you want to download
pdf_url = 'https://assets.openstax.org/oscms-prodcms/media/documents/ConceptsofBiology-WEB.pdf?_gl=1*pba6kg*_gcl_au*MTE2NjU5NDcyMi4xNzIzMzk1NDk5*_ga*Nzk1NjI1NjI4LjE3MjMzOTU1MDE.*_ga_T746F8B0QC*MTcyMzM5NTUwMC4xLjEuMTcyMzM5NTUxMi40OC4wLjA.'

# Download the PDF file
response = requests.get(pdf_url, verify = False)

# Save the PDF to the Colab filesystem
path = 'ConceptsofBiology-WEB.pdf'
with open(path, 'wb') as f:
    f.write(response.content)

print(f"PDF downloaded and saved to {path}")

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'assets.openstax.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


PDF downloaded and saved to ConceptsofBiology-WEB.pdf


In [7]:
#indexing two chapters of this book
start_page = 1
end_page = 68
texts = pdf_to_text(path, start_page=start_page, end_page=end_page)
chunks = text_to_chunks(texts, start_page=start_page)

In [8]:
print(chunks)

['[Page no. 4] "Concepts of Biology SENIOR CONTRIBUTING AUTHORS SAMANTHA FOWLER, CLAYTON STATE UNIVERSITY REBECCA ROUSH, SANDHILLS COMMUNITY COLLEGE JAMES WISE, HAMPTON UNIVERSITY  OpenStax Rice University 6100 Main Street MS-375 Houston, Texas 77005 To learn more about OpenStax, visit https://openstax.org. Individual print copies and bulk orders can be purchased"', '[Page no. 4] "https://openstax.org. Individual print copies and bulk orders can be purchased through our website. ©2023 Rice University. Textbook content produced by OpenStax is licensed under a Creative Commons Attribution 4.0 International License (CC BY 4.0). Under this license, any user of this textbook or the textbook contents herein must provide proper"', '[Page no. 4] "this textbook or the textbook contents herein must provide proper attribution as follows: - If you redistribute this textbook in a digital format (including but not limited to PDF and HTML), then you must retain on every page the following attribution

# 2. Embeddings and Vector Storage

In [9]:
# Load model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Compute embeddings
embeddings = model.encode(chunks)

# Create FAISS index
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)

# Add database vectors to the index
index.add(np.array(embeddings))

# Store chunk mappings
chunk_mapping = {i: chunk for i, chunk in enumerate(chunks)}
#print(chunk_mapping)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# 3. RAG Pipeline

In [10]:
# Load model
model_name = "t5-large"
model1 = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
def generate_answer(question):
    """
    Generate an answer to a given question based on relevant chunks of text.

    This function retrieves relevant chunks of text related to the given question,
    concatenates them with the question to form the input for a generative model,
    and produces an answer using that model.

    The function performs the following steps:
    1. Encodes the question into an embedding.
    2. Searches for the most similar text chunks based on the embedding.
    3. Concatenates the retrieved chunks and the question to form the input for
       the generative model.
    4. Generates an answer using the model and returns it.

    Parameters:
    question (str): The question for which an answer is to be generated.

    Returns:
    str: The generated answer in response to the question.
    """
    # Retrieve relevant chunks
    question_embedding = model.encode([question])
    D, I = index.search(np.array([question_embedding]).reshape(np.array([question_embedding]).shape[0],np.array([question_embedding]).shape[2]), k=5)
    retrieved_chunks = [chunk_mapping[i] for i in I[0]]

    # Generate answer
    input_text = " ".join(retrieved_chunks) + " " + question

    #input_text = "question:" + question + "context:" + " ".join(retrieved_chunks)
    input_text = "Question: " + question + "\nContext: " + " ".join(retrieved_chunks) + "\nAnswer:"

    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model1.generate(inputs, max_length=150, num_return_sequences=1)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [12]:
# Test the pipeline
question = "What is this book about?"
answer = generate_answer(question)
print("Answer:", answer)

Answer: Concepts of Biology


In [13]:
# List of pre-defined questions
questions = [
    "What is this book about?",
    "What is name of unit 1?",
    "What is name of unit 2?",
    "What are the key differences between prokaryotic and eukaryotic cells?"
]

# Function to handle custom and pre-defined questions
def handle_question(input_question, selected_question):
    """
    Handle and generate an answer for a question based on user input.

    This function determines the appropriate question to use based on whether a
    `selected_question` is provided. If a `selected_question` is provided, it is
    used as the question. Otherwise, the `input_question` is used. If no question
    is provided, the function returns a prompt asking the user to enter or select
    a question. Otherwise, it generates and returns an answer using the
    `generate_answer` function.

    Parameters:
    input_question (str): The question provided by the user as input.
    selected_question (str, optional): The question selected by the user from a list
                                        or options. If provided, it takes precedence
                                        over `input_question`.

    Returns:
    str: The generated answer in response to the selected or input question, or
         a prompt if no question is provided.
    """

    if selected_question:
        question = selected_question
    else:
        question = input_question

    if not question:
        return "Please enter or select a question."

    return generate_answer(question)

# Create Gradio interface
def create_interface():
    """
    Create a Gradio interface for question answering.

    This function sets up a Gradio interface that allows users to enter a question
    manually or select a pre-defined question from a dropdown menu. Users can submit
    their question by clicking a button, and the interface will display the generated
    answer in a non-interactive textbox.

    The interface includes:
    - A textbox for entering a question.
    - A dropdown menu for selecting from pre-defined questions.
    - A submit button to trigger the question handling and answer generation.
    - An output textbox to display the generated answer.

    Returns:
    gr.Blocks: A Gradio `Blocks` object representing the created interface.
    """
    with gr.Blocks() as demo:
        with gr.Row():
            question_textbox = gr.Textbox(label="Enter Your Question")
            question_dropdown = gr.Dropdown(choices=questions, label="Or Select a Pre-defined Question")
        with gr.Row():
            answer_output = gr.Textbox(label="Answer", interactive=False)
        submit_button = gr.Button("Submit")

        submit_button.click(
            fn=handle_question,
            inputs=[question_textbox, question_dropdown],
            outputs=answer_output
        )

    return demo

# Launch the Gradio app
interface = create_interface()
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e984bb991d6c60cb71.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
